# Combinando Datasets: Concat e Append

Alguns dos estudos mais interessantes de dados vêm da combinação de diferentes fontes de dados. Essas operações podem envolver qualquer coisa, desde concatenação muito direta de dois conjuntos de dados diferentes, até junções e mesclagens mais complicadas no estilo de banco de dados que lidam corretamente com qualquer sobreposição entre os conjuntos de dados. As `Series` e os `DataFrames` são criados com esse tipo de operação em mente, e o Pandas inclui funções e métodos que tornam esse tipo de dados rápida e direta.

In [1]:
import pandas as pd
import numpy as np

Por conveniência, definiremos essa função que cria um DataFrame de um formulário específico que será útil abaixo:

In [2]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


Além disso, criaremos uma classe rápida que nos permitirá exibir vários `DataFrames` lado a lado. O código utiliza o método `_repr_html_` especial, usado pelo IPython para implementar sua exibição de objeto avançado:

In [3]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

## Concatenação com Numpy Arrays

A concatenação dos objetos `Series` e `DataFrame` é muito semelhante à concatenação de matrizes `Numpy`, que pode ser feita através da função `np.concatenate`. O resultado é uma única matriz.

In [4]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

O primeiro argumento é uma lista ou tupla de matrizes para concatenar. Além disso, é necessária uma palavra-chave do eixo que permita especificar o eixo ao longo do qual o resultado será concatenado:

In [5]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

## Simples concatenação utilizando o método pd.concat

O Pandas tem uma função, `pd.concat()`, que possui uma sintaxe semelhante ao `np.concatenate` mas contém várias opções que discutiremos momentaneamente:

In [ ]:
# Signature in Pandas v0.18
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)


`pd.concat()` pode ser usado para uma concatenação simples de objetos `Series` ou `DataFrame`, assim como `np.concatenate()` pode ser usado para concatenações simples de matrizes:

In [6]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

Também funciona para concatenar objetos de maior dimensão, como `DataFrames`:

In [7]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


Por padrão, a concatenação ocorre em linhas no `DataFrame` (ou seja, eixo = 0). Como o `np.concatenate`, o `pd.concat` permite a especificação de um eixo ao longo do qual a concatenação ocorrerá. Considere o seguinte exemplo:

In [12]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
#pd.concat([df3, df4], axis=1)
display('df3', 'df4', "pd.concat([df3, df4], axis=1)")

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis=1)
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

Poderíamos ter especificado `axis = 'col'`;

## Índices Duplicados

Uma diferença importante entre `np.concatenate` e `pd.concat` é que a concatenação do Pandas preserva índices, mesmo que o resultado tenha índices duplicados! Considere este exemplo simples:

In [13]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


Observe os índices repetidos no resultado. Embora isso seja válido nos `DataFrames`, o resultado geralmente é indesejável. O `pd.concat()` nos oferece algumas maneiras de lidar com isso.

### Capturando as repetições como um erro

Se você quiser simplesmente verificar se os índices no resultado de `pd.concat()` não se sobrepõem, você pode especificar o sinalizador `verify_integrity`. Com esse conjunto como `True`, a concatenação gerará uma exceção se houver índices duplicados. Aqui está um exemplo, onde, para maior clareza, capturaremos e imprimiremos a mensagem de erro:

In [14]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


### Ignorando o índice

Às vezes, o próprio índice não importa e você prefere que seja simplesmente ignorado. Esta opção pode ser especificada usando o sinalizador `ignore_index`. Com esse conjunto como true, a concatenação criará um novo índice inteiro para a `Serie` resultante:

In [15]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


### Adicionando chaves MultiIndex

In [16]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

## Concatenação com joins

Nos exemplos simples que acabamos de ver, concatenamos principalmente `DataFrames` com nomes de colunas compartilhados. Na prática, dados de diferentes fontes podem ter diferentes conjuntos de nomes de colunas e o `pd.concat` oferece várias opções nesse caso. Considere a concatenação dos dois seguintes `DataFrames` que têm algumas (mas não todas!) Colunas em comum:

In [17]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

/home/jailsonpereira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

Por padrão, as entradas para as quais nenhum dado está disponível são preenchidas com valores de NaN. Para mudar isso, podemos especificar uma das várias opções para os parâmetros `join` e `join_axes` da função concatenate. Por padrão, a junção é uma união das colunas de entrada (join = 'outer'), mas podemos mudar isso para uma interseção das colunas usando join = 'inner':

In [18]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

Outra opção é especificar diretamente o índice das colunas restantes usando o argumento `join_axes`, que obtém uma lista de objetos de índice. Aqui, especificaremos que as colunas retornadas devem ser as mesmas da primeira entrada:

In [19]:
display('df5', 'df6',
        "pd.concat([df5, df6], join_axes=[df5.columns])")

,A,B,C
1,A1,B1,C1
2,A2,B2,C2
,B,C,D
3,B3,C3,D3
4,B4,C4,D4
,A,B,C
1,A1,B1,C1
2,A2,B2,C2
3,NaN,B3,C3
4,NaN,B4,C4


A combinação de opções da função `pd.concat` permite uma ampla gama de comportamentos possíveis ao unir dois conjuntos de dados; tenha isso em mente ao usar essas ferramentas para seus próprios dados.

## O append() método

Como a concatenação direta de matriz é muito comum, os objetos `Series` e `DataFrame` têm um método de acréscimo que pode realizar a mesma coisa com menos pressionamentos de tecla. Por exemplo, em vez de chamar pd.concat ([df1, df2]), você pode simplesmente chamar `df1.append (df2):